In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from scipy.sparse import coo_matrix

# ------------------------------------
# Configuration & Hyperparameters
# ------------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEEDS = [8]
GRAPH_SEED = 11

GRAPH_LEARN_MEAN = 0.5
GRAPH_LEARN_STD = 0.01
GRAPH_LEARN_EPOCHS = 5000
GRAPH_LEARN_ALPHA = 1.0
GRAPH_LEARN_BETA = 2.0
GRAPH_LEARN_ETA = 0.001

SAGE_HIDDEN_DIM = 128
SAGE_DROPOUT = 0.2
LR = 4e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 15000
PATIENCE = 1000

# ------------------------------------
# Learn Graph Structure
# ------------------------------------
def learn_graph(X: torch.Tensor, seed_graph: int) -> np.ndarray:
    X_np = X.cpu().numpy()
    N = X_np.shape[0]

    rng = np.random.default_rng(seed_graph)
    W = rng.normal(GRAPH_LEARN_MEAN, GRAPH_LEARN_STD, size=(N, N))

    i_idx, j_idx = np.triu_indices(N, k=1)
    w = W[i_idx, j_idx].copy()
    delL = np.sum((X_np[i_idx] - X_np[j_idx]) ** 2, axis=1)

    row = np.concatenate([i_idx, j_idx])
    col = np.concatenate([np.arange(len(i_idx)), np.arange(len(j_idx))])
    S = coo_matrix((np.ones(len(row)), (row, col)), shape=(N, len(i_idx))).tocsc()
    S_T = S.T

    best_w = w.copy()
    best_loss = np.inf

    for _ in range(GRAPH_LEARN_EPOCHS):
        Sw = S.dot(w)
        inv_Sw = np.zeros_like(Sw)
        nonzero = Sw > 0
        inv_Sw[nonzero] = 1.0 / Sw[nonzero]
        grad = 2.0 * delL + GRAPH_LEARN_BETA * w - GRAPH_LEARN_ALPHA * S_T.dot(inv_Sw)
        w -= GRAPH_LEARN_ETA * grad
        np.clip(w, 0.0, None, out=w)

        loss = (delL * w).sum() + (GRAPH_LEARN_BETA / 2.0) * (w ** 2).sum() - GRAPH_LEARN_ALPHA * np.sum(np.log(Sw[nonzero]))
        if loss < best_loss:
            best_loss = loss
            best_w = w.copy()

    num_edges = len(best_w)
    k = max(1, int(0.05 * num_edges))
    threshold = np.partition(best_w, -k)[-k]
    top_mask = best_w >= threshold

    adj = np.zeros((N, N), dtype=int)
    adj[i_idx[top_mask], j_idx[top_mask]] = 1
    adj[j_idx[top_mask], i_idx[top_mask]] = 1

    return adj

# ------------------------------------
# GNN Model
# ------------------------------------
class GraphSAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.out = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv3(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv4(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        return self.out(x, edge_index)

# ------------------------------------
# Train and Evaluate for One Seed
# ------------------------------------
def run_single_seed(X_raw, Y_raw, seed_model, seed_graph):
    np.random.seed(seed_model)
    torch.manual_seed(seed_model)
    if DEVICE.type == 'cuda':
        torch.cuda.manual_seed_all(seed_model)

    scaler = StandardScaler()
    X_np = scaler.fit_transform(np.abs(X_raw.cpu().numpy()))
    X = torch.tensor(X_np, dtype=torch.float32)

    adj = learn_graph(X, seed_graph)
    edge_index = torch.tensor(np.vstack(adj.nonzero()), dtype=torch.long)

    idx = np.arange(X.shape[0])
    train_idx, temp_idx = train_test_split(idx, test_size=0.2, random_state=seed_model)
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=seed_model)

    def get_mask(idxs):
        mask = torch.zeros(X.shape[0], dtype=torch.bool)
        mask[idxs] = True
        return mask

    data = Data(
        x=X.to(DEVICE),
        y=Y_raw.to(DEVICE),
        edge_index=edge_index.to(DEVICE)
    )
    masks = {k: get_mask(v).to(DEVICE) for k, v in zip(['train', 'val', 'test'], [train_idx, val_idx, test_idx])}

    model = GraphSAGENet(X.size(1), SAGE_HIDDEN_DIM, Y_raw.size(1)).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=200)
    criterion = torch.nn.MSELoss()

    best_val = np.inf
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[masks['train']], data.y[masks['train']])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(data.x, data.edge_index)[masks['val']], data.y[masks['val']])
        scheduler.step(val_loss)

        if val_loss < best_val:
            best_val = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break

    model.load_state_dict(best_state)
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        y_true = data.y

    r2 = r2_score(y_true[masks['test']].cpu().numpy(), out[masks['test']].cpu().numpy())
    mse = mean_squared_error(y_true[masks['test']].cpu().numpy(), out[masks['test']].cpu().numpy())

    return r2, mse

# ------------------------------------
# Main Execution
# ------------------------------------
if __name__ == "__main__":
    df = pd.read_csv("1500_cylindrical_anomalies.csv")
    X_raw = torch.tensor(df.iloc[:, :22].values, dtype=torch.float32)
    Y_raw = torch.tensor(df.iloc[:, 22:23].values, dtype=torch.float32)

    r2_score_val, mse_score_val = run_single_seed(
        X_raw, Y_raw,
        seed_model=SEEDS[0],
        seed_graph=GRAPH_SEED
    )
    print("Final Evaluation Metrics (R² and MSE) on Test Set:")
    print(f"R²: {r2_score_val:.4f}")
    print(f"MSE: {mse_score_val:.4f}")


Final Evaluation Metrics (R² and MSE) on Test Set:
R²: 0.9688
MSE: 0.1308
